In [1]:
import os 
import time
from tqdm import tqdm

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import wandb

from PIL import Image

## GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [2]:
#pip install wandb

In [3]:
os.getcwd()

'/home/jovyan'

In [4]:
### Custom Dataset ###
class CUB2011(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode

        if self.mode == 'train':
            self.image_folder = os.listdir('datasets/train')
        elif self.mode == 'valid':
            self.image_folder = os.listdir('datasets/valid')
        elif self.mode =='test':
            self.image_folder = os.listdir('datasets/test')
    
    def __len__(self):
        return len(self.image_folder)
    
    def __getitem__(self, idx):
        img_path = self.image_folder[idx]
        img = Image.open(os.path.join('datasets/', self.mode,img_path)).convert('RGB')
        img = self.transform(img)

        label = img_path.split('_')[-1].split('.')[0]
        label = int(label)
        return (img, label)

transforms_train = transforms.Compose([transforms.Resize((448,448)),
                                        transforms.ToTensor(),])

transforms_valtest = transforms.Compose([transforms.Resize((448,448)),
                                        transforms.ToTensor(),])

In [5]:
#Batch_size = 16

train_set = CUB2011(mode = 'train',
                    transform=transforms_train)
val_set = CUB2011(mode = 'valid',
                    transform=transforms_valtest)
test_set = CUB2011(mode = 'test',
                    transform=transforms_train)

print("Num of each dataset : ", len(train_set), len(val_set), len(test_set))

Num of each dataset :  2360 296 298


In [6]:
## Model / Optimizer ###
#EPOCH = 30
#lr = 0.1

model = models.resnet18(pretrained = True)

### Transfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 50)
model.to(DEVICE)

print("Created a learning model and optimizer")

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Created a learning model and optimizer


In [7]:
### Train / Evaluation ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    for i, (image,target) in enumerate (train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)
        
        train_loss.backward() 
        optimizer.step()
        
        if i % 10 ==0:
            print(f"Train_Epoch : {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item():.6f}")
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch})
    
    return train_loss

def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (image,target) in enumerate (val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)

            eval_loss += F.cross_entropy(output,
                                         target,reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    
    wandb.log({"val_loss": eval_loss, "val_accuracy": eval_accuracy})
    return eval_loss, eval_accuracy

In [8]:
sweep_config = {
    'name' : 'ActiveLearning 1',
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',  # 최적화할 메트릭
        'goal': 'maximize'  # 성능을 최대화하려는 목표
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.001]  # 실험할 learning rate
        },
        'epochs': {
            'values': [30,50]
        },
        'batch_size': {
            'values': [16, 32]  # 실험할 배치 사이즈
        },
        'optimizer': {
            'values': ['sgd', 'rmsprop', 'adam']  # 사용할 optimizer
        }
    }
}

# Sweep 생성
sweep_id = wandb.sweep(sweep_config, project="Active_Learning1")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 6gkqh4mq
Sweep URL: https://wandb.ai/empe8339-gachon-university/Active_Learning1/sweeps/6gkqh4mq


In [9]:
import wandb
import time
import torch.optim as optim
from tqdm import tqdm
import torch

def train_sweep():
    start = time.time()
    best = 0
    
    # wandb 설정 시작
    with wandb.init() as run:
        config = wandb.config
        
        train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=config.batch_size, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=config.batch_size, shuffle=False)
        
        if config.optimizer == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        elif config.optimizer == 'sgd':
            optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)
        elif config.optimizer == 'rmsprop':
            optimizer = optim.RMSprop(model.parameters(), lr=config.learning_rate)
        
        for epoch in tqdm(range(config.epochs), desc="Epochs"):
            epoch_start = time.time()
            train_loss = train(model, train_loader, optimizer, epoch)
            val_loss, val_accuracy = evaluate(model, val_loader)
            epoch_end = time.time()
            
            # 각 에포크에 대한 Computation Time, Memory 사용량 기록
            epoch_time = epoch_end - epoch_start
            if torch.cuda.is_available():
                memory_usage = torch.cuda.memory_allocated() / (1024 ** 3)  # GB 단위
            else:
                memory_usage = 0  # CPU 환경에서 사용할 경우
            
            # Save best Model
            if val_accuracy > best:
                best = val_accuracy
                torch.save(model.state_dict(), f"./Baseline/best_model_ActiveLearning#1.pth")
            
            # wandb 로그 추가
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'val_accuracy': val_accuracy,
                'epoch_time': epoch_time,  # Computation time
                'memory_usage_GB': memory_usage,  # Memory usage
            })

            print(f"[{epoch}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%, Epoch Time: {epoch_time:.2f}s, Memory Usage: {memory_usage:.2f}GB")
        
        # Test 결과 기록
        test_loss, test_accuracy = evaluate(model, test_loader)
        wandb.log({
            'test_loss': test_loss,
            'test_accuracy': test_accuracy
        })
        
        print(f"[Final] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%")
        
        end = time.time()
        elapsed_time = end - start
        print(f"Elapsed Time: {int(elapsed_time / 3600)}h, {int((elapsed_time % 3600) / 60)}m, {int(elapsed_time % 60)}s")

In [ ]:
# wandb sweep 실행
wandb.agent(sweep_id, function=train_sweep)

wandb: Agent Starting Run: nxnbddxt with config:
wandb: 	batch_size: 16
wandb: 	epochs: 30
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: Currently logged in as: empe8339 (empe8339-gachon-university). Use `wandb login --relogin` to force relogin


Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Train_Epoch : 0 [0/148]	Loss: 4.094913
Train_Epoch : 0 [10/148]	Loss: 3.971720
Train_Epoch : 0 [20/148]	Loss: 3.751534
Train_Epoch : 0 [30/148]	Loss: 3.870359
Train_Epoch : 0 [40/148]	Loss: 3.634727
Train_Epoch : 0 [50/148]	Loss: 3.701867
Train_Epoch : 0 [60/148]	Loss: 3.673403
Train_Epoch : 0 [70/148]	Loss: 3.634569
Train_Epoch : 0 [80/148]	Loss: 3.667145
Train_Epoch : 0 [90/148]	Loss: 3.251186
Train_Epoch : 0 [100/148]	Loss: 3.569783
Train_Epoch : 0 [110/148]	Loss: 2.978935
Train_Epoch : 0 [120/148]	Loss: 3.100534
Train_Epoch : 0 [130/148]	Loss: 2.903442
Train_Epoch : 0 [140/148]	Loss: 3.016335


Epochs:   3%|▎         | 1/30 [00:31<15:25, 31.93s/it]

[0] Validation Loss: 2.8683, Accuracy: 30.4054%, Epoch Time: 31.67s, Memory Usage: 0.08GB
Train_Epoch : 1 [0/148]	Loss: 2.666859
Train_Epoch : 1 [10/148]	Loss: 2.461011
Train_Epoch : 1 [20/148]	Loss: 2.824320
Train_Epoch : 1 [30/148]	Loss: 2.840235
Train_Epoch : 1 [40/148]	Loss: 2.606328
Train_Epoch : 1 [50/148]	Loss: 2.162285
Train_Epoch : 1 [60/148]	Loss: 2.185476
Train_Epoch : 1 [70/148]	Loss: 2.708738
Train_Epoch : 1 [80/148]	Loss: 2.075312
Train_Epoch : 1 [90/148]	Loss: 2.142694
Train_Epoch : 1 [100/148]	Loss: 2.190994
Train_Epoch : 1 [110/148]	Loss: 2.092141
Train_Epoch : 1 [120/148]	Loss: 1.840467
Train_Epoch : 1 [130/148]	Loss: 1.949438
Train_Epoch : 1 [140/148]	Loss: 1.917132


...

[11] Validation Loss: 3.2541, Accuracy: 46.2838%, Epoch Time: 29.50s, Memory Usage: 0.17GB
Train_Epoch : 12 [0/148]	Loss: 0.123997
Train_Epoch : 12 [10/148]	Loss: 0.157754
Train_Epoch : 12 [20/148]	Loss: 0.440949
Train_Epoch : 12 [30/148]	Loss: 0.109509
Train_Epoch : 12 [40/148]	Loss: 0.012659
Train_Epoch : 12 [50/148]	Loss: 0.095915
Train_Epoch : 12 [60/148]	Loss: 0.268032
Train_Epoch : 12 [70/148]	Loss: 0.195687
Train_Epoch : 12 [80/148]	Loss: 0.137467
Train_Epoch : 12 [90/148]	Loss: 0.346753
Train_Epoch : 12 [100/148]	Loss: 0.436785
Train_Epoch : 12 [110/148]	Loss: 0.024042
Train_Epoch : 12 [120/148]	Loss: 0.080054
Train_Epoch : 12 [130/148]	Loss: 0.306464
Train_Epoch : 12 [140/148]	Loss: 0.160672


Epochs:  43%|████▎     | 13/30 [06:22<08:19, 29.36s/it]

[12] Validation Loss: 3.4757, Accuracy: 46.2838%, Epoch Time: 29.22s, Memory Usage: 0.17GB
Train_Epoch : 13 [0/148]	Loss: 0.233946
Train_Epoch : 13 [10/148]	Loss: 0.021400
Train_Epoch : 13 [20/148]	Loss: 0.599954
Train_Epoch : 13 [30/148]	Loss: 0.100565
Train_Epoch : 13 [40/148]	Loss: 0.169519
Train_Epoch : 13 [50/148]	Loss: 0.184585
Train_Epoch : 13 [60/148]	Loss: 0.151684
Train_Epoch : 13 [70/148]	Loss: 0.229517
Train_Epoch : 13 [80/148]	Loss: 0.190980
Train_Epoch : 13 [90/148]	Loss: 0.429176
Train_Epoch : 13 [100/148]	Loss: 0.376785
Train_Epoch : 13 [110/148]	Loss: 0.358120
Train_Epoch : 13 [120/148]	Loss: 0.092068
Train_Epoch : 13 [130/148]	Loss: 0.062335
Train_Epoch : 13 [140/148]	Loss: 0.181306


Epochs:  47%|████▋     | 14/30 [06:51<07:49, 29.37s/it]

[13] Validation Loss: 3.6428, Accuracy: 44.2568%, Epoch Time: 29.39s, Memory Usage: 0.17GB
Train_Epoch : 14 [0/148]	Loss: 0.148313
Train_Epoch : 14 [10/148]	Loss: 0.139615
Train_Epoch : 14 [20/148]	Loss: 0.133390
Train_Epoch : 14 [30/148]	Loss: 0.157805
Train_Epoch : 14 [40/148]	Loss: 0.160663
Train_Epoch : 14 [50/148]	Loss: 0.867028
Train_Epoch : 14 [60/148]	Loss: 0.420414
